In [11]:
import pandas as pd
from tkinter import *
from tkinter import messagebox
from tkinter.ttk import Combobox
from tkinter import Spinbox
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.multioutput import MultiOutputRegressor

In [12]:
df = pd.read_csv('odyometri_test_sonuclari.csv')

In [13]:
label_encoder = LabelEncoder()
df['CİNSİYET'] = label_encoder.fit_transform(df['CİNSİYET'])

In [14]:
X = df[['YAŞ', 'CİNSİYET', 'R500Hz', 'R1000Hz', 'R2000Hz', 'R4000Hz', 'L500Hz', 'L1000Hz', 'L2000Hz', 'L4000Hz']]
y = df[['R_Safses_ortalama', 'L_Safses_ortalama']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

svr_model = SVR(C=1.0, kernel='rbf', epsilon=0.1)

multioutput_svr_model = MultiOutputRegressor(svr_model)

multioutput_svr_model.fit(X_train, y_train)
         

MultiOutputRegressor(estimator=SVR())

In [15]:
def hesapla():
    try:
    
        secilen_yas = int(yas_kutu.get())
        secilen_cinsiyet = cinsiyet_kutu.get()

        
        if secilen_cinsiyet == "Erkek":
            cinsiyet_kodu = 1
        elif secilen_cinsiyet == "Kadın":
            cinsiyet_kodu = 0
        else:
            raise ValueError("Cinsiyet bilgisi hatalı")

        
        yeni_veri = pd.DataFrame({
            'YAŞ': [secilen_yas],
            'CİNSİYET': [cinsiyet_kodu],
            'R500Hz': [0],
            'R1000Hz': [0],
            'R2000Hz': [0],
            'R4000Hz': [0],
            'L500Hz': [0],
            'L1000Hz': [0],
            'L2000Hz': [0],
            'L4000Hz': [0],
        })

        
        tahminler = multioutput_svr_model.predict(yeni_veri)

        
        messagebox.showinfo(title="Tahmin Sonuçları", message=f"Sağ ve Sol İşitme Kaybı Tahmini: {tahminler}")

    except ValueError as e:
        messagebox.showwarning(title="Hata", message=str(e))


In [16]:
pencere = Tk()
pencere.title("İşitme Kaybı Derece Tahmini")
pencere.configure(background='#9E0800')
pencere.geometry("800x600")  

''

In [17]:
baslik_label = Label(pencere, text="İşitme Kaybı Derecesi Tahmini", font="helvetica 18 bold",
                     background="#9E0800", padx=10, pady=10,foreground="black")
baslik_label.pack()

In [18]:
# Cinsiyet seçim kutucuğu
cinsiyet_label = Label(pencere, text="Cinsiyet", font="helvetica 12")
cinsiyet_label.pack()

cinsiyetler = ['Erkek', 'Kadın']
cinsiyet_kutu = Combobox(pencere, values=cinsiyetler)
cinsiyet_kutu.pack(pady=5)

In [19]:
# Yaş seçim kutucuğu
yas_label = Label(pencere, text="Yaş", font="helvetica 12")
yas_label.pack()

yas_kutu = Spinbox(pencere, from_=18, to=100)
yas_kutu.pack(pady=5) 

In [20]:
# Hesapla butonu
tahmin_buton = Button(pencere, text="Tahmin Yap", command=hesapla)
tahmin_buton.pack(pady=10)
# İşitme kaybı dereceleri bilgisi etiketi
hearing_loss_info = (
    "Sağ ve Sol İşitme Kaybı Ortalama Değerlerine göre ; \n \n \n"
    
    "Çok Hafif Dereceli İşitme Kaybı 16-25 dB\n \n \n"
    "Hafif Dereceli İşitme Kaybı 26-41 dB\n \n \n"
    
    "Orta Dereceli İşitme Kaybı 41-55 dB\n \n \n"
    
    "Orta-İleri Dereceli İşitme Kaybı 56-70 dB\n \n \n"
    
    " İleri Dereceli İşitme Kaybı 71-90 dB\n \n \n "
    
    " Çok İleri Dereceli İşitme Kaybı 90-120 dB\n \n \n "
    
    "Aralığına denk gelmektedir. Çıkan rakam değerlerinizi bu aralıklardan kontrol edebilir,işitme kaybı derecenizi öğrenebilirsiniz."
)
info_label = Label(pencere, text=hearing_loss_info, justify="left", padx=10, pady=10 ,font=("Arial", 10))
info_label.pack()

pencere.mainloop()